In [142]:
import attr
import cattr
import json
from pathlib import Path
import typing

In [143]:
@attr.s(frozen=True, kw_only=True)
class BarelyDBConfig:
    
    path_depth = attr.ib(default=0)
    known_bases = attr.ib(factory=list, type=typing.List[str])
    ignored_files = attr.ib(factory=list, type=typing.List[str])
    auto_reload_components = attr.ib(default=None)
    buid_types = attr.ib(factory=dict, type=typing.Dict[str, str])
    
    @classmethod
    def from_default(cls):
        return cls(
            path_depth = 1,
            known_bases = [\
        'G:\\My Drive\\Battrion_AG\\DATABASE\\',
        'G:\\Team Drives\\Database\\',
        'G:\\Shared drives\\Database\\',
        'G:\\Geteilte Ablagen\\Database\\',
        'G:\\Drive partagés\\Database\\',
        '/home/pi/GoogleDrive/database/',
        '/home/jovyan/database/',
        'barelydb://',
        'barely-db://',
        ],
        ignored_files = ['desktop.ini'],
        auto_reload_components = None,
        buid_types = {\
            'slurry': 'SL',
            'web': 'WB',
            'cells': 'CL',
            'electrochemistry': 'EE',
            'rawmaterial': 'RM',
            'experiment': 'EXP',
            'equipment': 'EQ',
            'manufacturing_orders': 'MO',
            'product': 'PD',
            'documents': 'DOC',
        }                   
                  )
    
    @staticmethod
    def resolve_config_file(base_path):
        base_path = Path(base_path)
        
        if base_path.is_dir():
            config_file = base_path.joinpath('bdb_config.json')
        else:
            config_file = base_path

        return str(config_file)

    def save(self, base_path):
        config_file = self.resolve_config_file(base_path)

        self_dict = attr.asdict(self)

        with open(config_file, 'w') as f:
            json.dump(self_dict, f, indent=4)

    @classmethod
    def load(cls, base_path):
        config_file = Path(cls.resolve_config_file(base_path))

        with open(config_file, 'rb') as f:
            self_dict = json.load(f)

        return cattr.structure(self_dict, cls)            
            
    

In [144]:
### Test restructuring
config = BarelyDBConfig()

self_dict = cattr.unstructure(config)
self_dict

config2 = cattr.structure(self_dict, BarelyDBConfig)
config2

BarelyDBConfig(path_depth=0, known_bases=[], ignored_files=[], auto_reload_components=None, buid_types={})

In [145]:
base_path_raw = 'G:\\Shared drives\\Database'
base_path_raw = 'C:\\Repo\\barely-db\\tests\\Database'
base_path = str(Path(base_path_raw))

In [146]:
config = BarelyDBConfig.from_default()
config

BarelyDBConfig(path_depth=1, known_bases=['G:\\My Drive\\Battrion_AG\\DATABASE\\', 'G:\\Team Drives\\Database\\', 'G:\\Shared drives\\Database\\', 'G:\\Geteilte Ablagen\\Database\\', 'G:\\Drive partagés\\Database\\', '/home/pi/GoogleDrive/database/', '/home/jovyan/database/', 'barelydb://', 'barely-db://'], ignored_files=['desktop.ini'], auto_reload_components=None, buid_types={'slurry': 'SL', 'web': 'WB', 'cells': 'CL', 'electrochemistry': 'EE', 'rawmaterial': 'RM', 'experiment': 'EXP', 'equipment': 'EQ', 'manufacturing_orders': 'MO', 'product': 'PD', 'documents': 'DOC'})

In [148]:
### Test save and reload
config.save(base_path=base_path)

config2 = BarelyDBConfig.load(base_path)
assert config2 == config